## Chia Pool Design Principles
- The farmer always hold their keys and never give keys to anyone.
- Plots are tied to a [singleton](../singleton/notebook.ipynb) **smart coin** which is pointed to pool. A smart coin is responsible for spliting rewards (1.75 to the pool and 0.25 to the farmer).
- Block creation is separated and can be done by either a farmer (if they run a full node) or pool.
- The farmer can switch pool any time without permission from anyone.
- Pool information is kept on blockchain.

## What's in the Plots?
### Original plots
1. Farmer public key, `-f`
2. Pool public key, `-p`

### Pooling plots (1.2+)

1. Farmer public key, `-f`
2. Pool contract address (AKA pay to singleton address), `-c`

<img src="https://docs.chia.net/img/Pooling_absorb.png" alt="Collecting pool rewards" width="600"/>

### p2_singleton_puzzle
- `p2_singleton_puzzle` (or its puzzle hash) is committed into the plot.
- The puzzle hash (or the address) receives the funds. The plot has an address and when the block is farmed, 1.75 goes to the address.
- 1.75 is not going to the user or to the pool. It has to be claimed later **to the pool**.
- To claim the 1.75, the singleton and the `p2_singleton_puzzle` coins have to be spent at the same time.
- Anyone can spend the singleton and `p2_singleton_puzzle`. This is called **absorb** transaction.
- [How do pool collect rewards?](https://github.com/Chia-Network/chia-blockchain/wiki/Pooling-FAQ#how-do-pool-collect-rewards)
    1. Farmer joins a pool, they will **assign their singleton to the pool_puzzle_hash** (`Pooling to: pool_addr`). 
    2. When a farmer wins a block, the pool rewards will be sent to the `p2_singleton_puzzle_hash`.
    3. Pool will scan blockchain to find new rewards sent to **Farmer's singletons** (pool has a launcher id).
    4. The pool will **send a request to claim rewards** (absorb) to the winning Farmer's singleton.
    5. Farmer's singleton will send pool rewards XCH to `pool_puzzle_hash`.
    6. Pool will periodically distribute rewards to farmers that have points.

### Difficulty
> As difficulty goes up, a farmer does less lookups and finds less proofs.
- [Pooling Difficulty](https://docs.chia.net/docs/11pooling/pooling#difficulty)
    > The difficulty allows the pool operator to control how many partials per day they are receiving from each farmer.

- Farmer gets the signage point from the blockchain and sends very low difficulty (like 1-10) to the harvester, so it finds a lot more proofs.
- Instead of sending proofs to the blockchain, it sends the winning blocks to the pool server. This is called **partials**.

### Partials
- [Pooling Partials](https://docs.chia.net/docs/11pooling/pooling#receiving-partials)
    > A partial is a proof of space with some additional metadata and authentication info from the farmer, which meets certain minimum difficulty requirements. Partials must be real proofs of space responding to blockchain signage points, and they must be submitted within the blockchain time window (~28 seconds after the signage point).

> **partials** is a proof of space with the very low difficulty (it's not good enough for the blockchain which has 2k+ difficulty).

### Points


## PlotNFT
0. Generate new key and add one mojo.
```sh
Fingerprint: 1073621827
Master public key (m): 83ed6407c1333464235a44a67747697a69629fd8465e61ac4aec21fe87ce877dc45b334ec1cef048aa6c3202ace9cbbe
Farmer public key (m/12381/8444/0/0): a188f3d225bfdf559dba021a6d1a320cf3b16b3fc241a490c954e09399a6bc117127e8c0527dbe9cc150b2e717f757d8
Pool public key (m/12381/8444/1/0): a8c1e86e701b130511814c3deaf860405648fb2616592bf2cb12b2cf69de4af9d8602c1bb40107ebd41c8338351051f2
First wallet address: txch1wgkkag72vvmawj5zy08rf2ts8vuuh0e0srlzzjr49p0krm7qhc2q0ezl6t
```

```sh
❯ chia wallet show -f 1073621827
Wallet height: 1022607
Sync status: Synced
Balances, fingerprint: 1073621827

Chia Wallet:
   -Total Balance:         1e-12 txch (1 mojo)
   -Pending Total Balance: 1e-12 txch (1 mojo)
   -Spendable:             1e-12 txch (1 mojo)
   -Type:                  STANDARD_WALLET
   -Wallet ID:             1
```

1. Create a PlotNFT (pool singleton).

- [plotnft_funcs.create](https://github.com/Chia-Network/chia-blockchain/blob/ab1ac66baaaf4ef3b87bd658465910967bef12ee/chia/cmds/plotnft_funcs.py#L56)
- [wallet_rpc_client.create_new_pool_wallet](https://github.com/Chia-Network/chia-blockchain/blob/ab1ac66baaaf4ef3b87bd658465910967bef12ee/chia/rpc/wallet_rpc_client.py#L283)
- [wallet_rpc_api.create `pool_wallet`](https://github.com/Chia-Network/chia-blockchain/blob/ab1ac66baaaf4ef3b87bd658465910967bef12ee/chia/rpc/wallet_rpc_api.py#L545)
- [pool_wallet.create_new_pool_wallet_transaction](https://github.com/Chia-Network/chia-blockchain/blob/ab1ac66baaaf4ef3b87bd658465910967bef12ee/chia/pools/pool_wallet.py#L389)

> A "plot NFT", or pool wallet, represents the idea of a set of plots that all pay to the same pooling puzzle. This puzzle is a `chia singleton` that is parameterized with a public key controlled by the user's wallet (a `smart coin`). It contains an inner puzzle that can switch between paying block rewards to a pool, or to a user's own wallet.

- [pool_wallet.generate_launcher_spend](https://github.com/Chia-Network/chia-blockchain/blob/ab1ac66baaaf4ef3b87bd658465910967bef12ee/chia/pools/pool_wallet.py#L619)

> Creates the initial singleton, which includes spending an origin coin, the launcher, and creating a singleton with the "pooling" inner state, which can be either self pooling or using a pool

- [pool_puzzles.create_pooling_inner_puzzle](https://github.com/Chia-Network/chia-blockchain/blob/ab1ac66baaaf4ef3b87bd658465910967bef12ee/chia/pools/pool_puzzles.py#L55)


#### [pool_member_innerpuz.clvm](https://github.com/Chia-Network/chia-blockchain/blob/ab1ac66baaaf4ef3b87bd658465910967bef12ee/chia/wallet/puzzles/pool_member_innerpuz.clvm)

```clojure
(mod (POOL_PUZZLE_HASH
      P2_SINGLETON_PUZZLE_HASH
      OWNER_PUBKEY
      POOL_REWARD_PREFIX
      WAITINGROOM_PUZHASH
      Truths
      p1
      pool_reward_height
      )


  ; POOL_PUZZLE_HASH is commitment to the pool's puzzle hash
  ; P2_SINGLETON_PUZZLE_HASH is the puzzle hash for your pay to singleton puzzle
  ; OWNER_PUBKEY is the farmer pubkey which authorises a travel
  ; POOL_REWARD_PREFIX is network-specific data (mainnet vs testnet) that helps determine if a coin is a pool reward
  ; WAITINGROOM_PUZHASH is the puzzle_hash you'll go to when you iniate the leaving process

  ; Absorbing money if pool_reward_height is an atom
  ; Escaping if pool_reward_height is ()

  ; p1 is pool_reward_amount if absorbing money
  ; p1 is extra_data key_value_list if escaping

  ; pool_reward_amount is the value of the coin reward - this is passed in so that this puzzle will still work after halvenings
  ; pool_reward_height is the block height that the reward was generated at. This is used to calculate the coin ID.
  ; key_value_list is signed extra data that the wallet may want to publicly announce for syncing purposes

  (include condition_codes.clvm)
  (include singleton_truths.clib)

  ; takes a lisp tree and returns the hash of it
  (defun sha256tree (TREE)
      (if (l TREE)
          (sha256 2 (sha256tree (f TREE)) (sha256tree (r TREE)))
          (sha256 1 TREE)
      )
  )

  (defun-inline calculate_pool_reward (pool_reward_height P2_SINGLETON_PUZZLE_HASH POOL_REWARD_PREFIX pool_reward_amount)
    (sha256 (logior POOL_REWARD_PREFIX (logand (- (lsh (q . 1) (q . 128)) (q . 1)) pool_reward_height)) P2_SINGLETON_PUZZLE_HASH pool_reward_amount)
  )

  (defun absorb_pool_reward (POOL_PUZZLE_HASH my_inner_puzzle_hash my_amount pool_reward_amount pool_reward_id)
    (list
        (list CREATE_COIN my_inner_puzzle_hash my_amount)
        (list CREATE_COIN POOL_PUZZLE_HASH pool_reward_amount)
        (list CREATE_PUZZLE_ANNOUNCEMENT pool_reward_id)
        (list ASSERT_COIN_ANNOUNCEMENT (sha256 pool_reward_id '$'))
    )
  )

  (defun-inline travel_to_waitingroom (OWNER_PUBKEY WAITINGROOM_PUZHASH my_amount extra_data)
    (list (list AGG_SIG_ME OWNER_PUBKEY (sha256tree extra_data))
          (list CREATE_COIN WAITINGROOM_PUZHASH my_amount)
    )
  )

  ; main

  (if pool_reward_height
    (absorb_pool_reward POOL_PUZZLE_HASH
                        (my_inner_puzzle_hash_truth Truths)
                        (my_amount_truth Truths)
                        p1
                        (calculate_pool_reward pool_reward_height P2_SINGLETON_PUZZLE_HASH POOL_REWARD_PREFIX p1)
    )
    (travel_to_waitingroom OWNER_PUBKEY WAITINGROOM_PUZHASH (my_amount_truth Truths) p1)
    )
  )
)
```




```sh
❯ chia plotnft create -f 1073621827 -s local
❯ chia wallet show -f 1073621827
Wallet height: 1022621
Sync status: Synced
Balances, fingerprint: 1073621827

Chia Wallet:
   -Total Balance:         0.0 txch (0 mojo)
   -Pending Total Balance: 0.0 txch (0 mojo)
   -Spendable:             0.0 txch (0 mojo)
   -Type:                  STANDARD_WALLET
   -Wallet ID:             1

Pool wallet:
   -Total Balance:         0.0 txch (0 mojo)
   -Pending Total Balance: 0.0 txch (0 mojo)
   -Spendable:             0.0 txch (0 mojo)
   -Type:                  POOLING_WALLET
   -Wallet ID:             2
```

2. Inspect the PlotNFT.

```sh
❯ chia plotnft inspect -f 1073621827 --id 2
{'pool_wallet_info': PoolWalletInfo(current=PoolState(version=1, state=1, target_puzzle_hash=<bytes32: b973065072282d2eab48e40d075fba0a081b72f0f2a30acf40af2030d4fa9fd7>, owner_pubkey=<G1Element 92a5174d928c42acf3ef59008745ee7b6fb6f27ce614f373d9c007e013d74abc73885af0cc7729a8c72cf0b8b6fa8407>, pool_url='', relative_lock_height=0), target=None, launcher_coin=Coin(parent_coin_info=<bytes32: 59080c1774d6d27520dc5ef68977b15c33ae6a75102b0a10b27d235d4ed31e32>, puzzle_hash=<bytes32: eff07522495060c066f66f32acc2a77e3a3e737aca8baea4d1a64ea4cdc13da9>, amount=1), launcher_id=<bytes32: ab2b05b64057883971197d524569f96313656ffe213d63879a0ce019505f0a60>, p2_singleton_puzzle_hash=<bytes32: d600f7959c8073a79a90b58016e1a32568485e8ac31eca047b808db0869a11d2>, current_inner=Program(ff02ffff01ff02ffff01ff02ffff03ff82017fffff01ff04ffff04ff1cffff04ff5fff808080ffff04ffff04ff12ffff04ff8205ffffff04ff8206bfff80808080ffff04ffff04ff08ffff04ff17ffff04ffff02ff1effff04ff02ffff04ffff04ff8205ffffff04ff8202ffff808080ff80808080ff80808080ff80808080ffff01ff02ff16ffff04ff02ffff04ff05ffff04ff8204bfffff04ff8206bfffff04ff8202ffffff04ffff0bffff19ff2fffff18ffff019100ffffffffffffffffffffffffffffffffff8205ff8080ff0bff8202ff80ff808080808080808080ff0180ffff04ffff01ffff32ff3d52ffff333effff04ffff04ff12ffff04ff0bffff04ff17ff80808080ffff04ffff04ff12ffff04ff05ffff04ff2fff80808080ffff04ffff04ff1affff04ff5fff808080ffff04ffff04ff14ffff04ffff0bff5fffff012480ff808080ff8080808080ff02ffff03ffff07ff0580ffff01ff0bffff0102ffff02ff1effff04ff02ffff04ff09ff80808080ffff02ff1effff04ff02ffff04ff0dff8080808080ffff01ff0bffff0101ff058080ff0180ff018080ffff04ffff01a0b973065072282d2eab48e40d075fba0a081b72f0f2a30acf40af2030d4fa9fd7ffff04ffff01a0d600f7959c8073a79a90b58016e1a32568485e8ac31eca047b808db0869a11d2ffff04ffff01b092a5174d928c42acf3ef59008745ee7b6fb6f27ce614f373d9c007e013d74abc73885af0cc7729a8c72cf0b8b6fa8407ffff04ffff01a0ae83525ba8d1dd3f09b277de18ca3e4300000000000000000000000000000000ffff04ffff0180ff01808080808080), tip_singleton_coin_id=<bytes32: dbb435bfa82340a04d7f3ad6bbf534cfd8a6852e015f8c4f652c7f6bad83c4ff>, singleton_block_height=1022620), 'unconfirmed_transactions': []}
```

```sh
❯ chia plotnft show -f 1073621827
Wallet height: 1022936
Sync status: Synced
Wallet id 2:
Current state: SELF_POOLING
Current state from block height: 1022620
Launcher ID: ab2b05b64057883971197d524569f96313656ffe213d63879a0ce019505f0a60
Target address (not for plotting): txch1h9esv5rj9qkja26gusxswha6pgypkuhs723s4n6q4usrp486nltsjp3q0u
Number of plots: 0
Owner public key: 92a5174d928c42acf3ef59008745ee7b6fb6f27ce614f373d9c007e013d74abc73885af0cc7729a8c72cf0b8b6fa8407
Pool contract address (use ONLY for plotting - do not send money to this address): txch16cq009vuspe60x5skkqpdcdry45ysh52cv0v5prmszxmpp56z8fqgepz05
Claimable balance: 0.0 txch (0 mojo)
```

In [1]:
%%bash
# find the standard coin that we use to create the PlotNFT
# /get_coin_records_by_puzzle_hash
cdv decode txch1wgkkag72vvmawj5zy08rf2ts8vuuh0e0srlzzjr49p0krm7qhc2q0ezl6t
curl --insecure --cert ~/.chia/testnet/config/ssl/full_node/private_full_node.crt \
--key ~/.chia/testnet/config/ssl/full_node/private_full_node.key \
-H "Accept: application/json" \
-H "Content-Type: application/json" \
-X POST https://host.docker.internal:8555/get_coin_records_by_puzzle_hash -s \
-d '{"puzzle_hash": "0x722d6ea3ca6337d74a8223ce34a9703b39cbbf2f80fe214875285f61efc0be14", "include_spent_coins": true}' | jq --color-output

722d6ea3ca6337d74a8223ce34a9703b39cbbf2f80fe214875285f61efc0be14
{
  "coin_records": [
    {
      "coin": {
        "amount": 1,
        "parent_coin_info": "0x1b03cf435594f6b101a8bac65fdad5099be9dd14400ed0b3c21340e8839b7e8a",
        "puzzle_hash": "0x722d6ea3ca6337d74a8223ce34a9703b39cbbf2f80fe214875285f61efc0be14"
      },
      "coinbase": false,
      "confirmed_block_index": 1022604,
      "spent": true,
      "spent_block_index": 1022620,
      "timestamp": 1653467044
    }
  ],
  "success": true
}


In [13]:
%%bash
# get coin id
run '(sha256 0x1b03cf435594f6b101a8bac65fdad5099be9dd14400ed0b3c21340e8839b7e8a 0x722d6ea3ca6337d74a8223ce34a9703b39cbbf2f80fe214875285f61efc0be14 1)'

0x59080c1774d6d27520dc5ef68977b15c33ae6a75102b0a10b27d235d4ed31e32


In [2]:
%%bash
# get the launcher coin
# /get_coin_records_by_parent_ids
curl --insecure --cert ~/.chia/testnet/config/ssl/full_node/private_full_node.crt \
--key ~/.chia/testnet/config/ssl/full_node/private_full_node.key \
-H "Accept: application/json" \
-H "Content-Type: application/json" \
-X POST https://host.docker.internal:8555/get_coin_records_by_parent_ids -s \
-d '{"parent_ids": ["0x59080c1774d6d27520dc5ef68977b15c33ae6a75102b0a10b27d235d4ed31e32"], "include_spent_coins": true}' | jq --color-output

{
  "coin_records": [
    {
      "coin": {
        "amount": 1,
        "parent_coin_info": "0x59080c1774d6d27520dc5ef68977b15c33ae6a75102b0a10b27d235d4ed31e32",
        "puzzle_hash": "0xeff07522495060c066f66f32acc2a77e3a3e737aca8baea4d1a64ea4cdc13da9"
      },
      "coinbase": false,
      "confirmed_block_index": 1022620,
      "spent": true,
      "spent_block_index": 1022620,
      "timestamp": 1653467258
    }
  ],
  "success": true
}


In [15]:
%%bash
# get the launcher coin id
# this is the launcher id
run '(sha256 0x59080c1774d6d27520dc5ef68977b15c33ae6a75102b0a10b27d235d4ed31e32 0xeff07522495060c066f66f32acc2a77e3a3e737aca8baea4d1a64ea4cdc13da9 1)'

0xab2b05b64057883971197d524569f96313656ffe213d63879a0ce019505f0a60


In [3]:
%%bash
# get the puzzle reveal and solution when the launcher coin is spent
# /get_puzzle_and_solution
# height is the spent_block_index
curl --insecure --cert ~/.chia/testnet/config/ssl/full_node/private_full_node.crt \
--key ~/.chia/testnet/config/ssl/full_node/private_full_node.key \
-H "Accept: application/json" \
-H "Content-Type: application/json" \
-X POST https://host.docker.internal:8555/get_puzzle_and_solution -s \
-d '{"coin_id": "0xab2b05b64057883971197d524569f96313656ffe213d63879a0ce019505f0a60", "height": 1022620}' | jq --color-output

{
  "coin_solution": {
    "coin": {
      "amount": 1,
      "parent_coin_info": "0x59080c1774d6d27520dc5ef68977b15c33ae6a75102b0a10b27d235d4ed31e32",
      "puzzle_hash": "0xeff07522495060c066f66f32acc2a77e3a3e737aca8baea4d1a64ea4cdc13da9"
    },
    "puzzle_reveal": "0xff02ffff01ff04ffff04ff04ffff04ff05ffff04ff0bff80808080ffff04ffff04ff0affff04ffff02ff0effff04ff02ffff04ffff04ff05ffff04ff0bffff04ff17ff80808080ff80808080ff808080ff808080ffff04ffff01ff33ff3cff02ffff03ffff07ff0580ffff01ff0bffff0102ffff02ff0effff04ff02ffff04ff09ff80808080ffff02ff0effff04ff02ffff04ff0dff8080808080ffff01ff0bffff0101ff058080ff0180ff018080",
    "solution": "0xffa07fb5cdb5f02bd956147cca3e4420dce2cac324d44840eae544a7f42e2c1ed124ff01ffffff70c05b0101b973065072282d2eab48e40d075fba0a081b72f0f2a30acf40af2030d4fa9fd792a5174d928c42acf3ef59008745ee7b6fb6f27ce614f373d9c007e013d74abc73885af0cc7729a8c72cf0b8b6fa8407010000000000000000ffff7483093a80ffff68a06c4b2309a44a91ebc1621d79bcbbd4125dd271ee5ecc9bc4553f12d085b35872808

In [22]:
%%bash
# disassemble puzzle_reveal and solution
opd ff02ffff01ff04ffff04ff04ffff04ff05ffff04ff0bff80808080ffff04ffff04ff0affff04ffff02ff0effff04ff02ffff04ffff04ff05ffff04ff0bffff04ff17ff80808080ff80808080ff808080ff808080ffff04ffff01ff33ff3cff02ffff03ffff07ff0580ffff01ff0bffff0102ffff02ff0effff04ff02ffff04ff09ff80808080ffff02ff0effff04ff02ffff04ff0dff8080808080ffff01ff0bffff0101ff058080ff0180ff018080
opd ffa07fb5cdb5f02bd956147cca3e4420dce2cac324d44840eae544a7f42e2c1ed124ff01ffffff70c05b0101b973065072282d2eab48e40d075fba0a081b72f0f2a30acf40af2030d4fa9fd792a5174d928c42acf3ef59008745ee7b6fb6f27ce614f373d9c007e013d74abc73885af0cc7729a8c72cf0b8b6fa8407010000000000000000ffff7483093a80ffff68a06c4b2309a44a91ebc1621d79bcbbd4125dd271ee5ecc9bc4553f12d085b358728080

(a (q 4 (c 4 (c 5 (c 11 ()))) (c (c 10 (c (a 14 (c 2 (c (c 5 (c 11 (c 23 ()))) ()))) ())) ())) (c (q 51 60 2 (i (l 5) (q 11 (q . 2) (a 14 (c 2 (c 9 ()))) (a 14 (c 2 (c 13 ())))) (q 11 (q . 1) 5)) 1) 1))
(0x7fb5cdb5f02bd956147cca3e4420dce2cac324d44840eae544a7f42e2c1ed124 1 ((112 . 0x0101b973065072282d2eab48e40d075fba0a081b72f0f2a30acf40af2030d4fa9fd792a5174d928c42acf3ef59008745ee7b6fb6f27ce614f373d9c007e013d74abc73885af0cc7729a8c72cf0b8b6fa8407010000000000000000) (116 . 0x093a80) (104 . 0x6c4b2309a44a91ebc1621d79bcbbd4125dd271ee5ecc9bc4553f12d085b35872)))


In [24]:
%%bash
# run the puzzle and get conditions
brun '(a (q 4 (c 4 (c 5 (c 11 ()))) (c (c 10 (c (a 14 (c 2 (c (c 5 (c 11 (c 23 ()))) ()))) ())) ())) (c (q 51 60 2 (i (l 5) (q 11 (q . 2) (a 14 (c 2 (c 9 ()))) (a 14 (c 2 (c 13 ())))) (q 11 (q . 1) 5)) 1) 1))' \
'(0x7fb5cdb5f02bd956147cca3e4420dce2cac324d44840eae544a7f42e2c1ed124 1 ((112 . 0x0101b973065072282d2eab48e40d075fba0a081b72f0f2a30acf40af2030d4fa9fd792a5174d928c42acf3ef59008745ee7b6fb6f27ce614f373d9c007e013d74abc73885af0cc7729a8c72cf0b8b6fa8407010000000000000000) (116 . 0x093a80) (104 . 0x6c4b2309a44a91ebc1621d79bcbbd4125dd271ee5ecc9bc4553f12d085b35872)))'

((51 0x7fb5cdb5f02bd956147cca3e4420dce2cac324d44840eae544a7f42e2c1ed124 1) (60 0x55c9f41c0195070b524cbab9146d999d93286937fdc7ac9a7c066b99b3371ec1))


In [4]:
%%bash
# get the eve singleton
curl --insecure --cert ~/.chia/testnet/config/ssl/full_node/private_full_node.crt \
--key ~/.chia/testnet/config/ssl/full_node/private_full_node.key \
-H "Accept: application/json" \
-H "Content-Type: application/json" \
-X POST https://host.docker.internal:8555/get_coin_records_by_parent_ids -s \
-d '{"parent_ids": ["0xab2b05b64057883971197d524569f96313656ffe213d63879a0ce019505f0a60"], "include_spent_coins": true}' | jq --color-output

{
  "coin_records": [
    {
      "coin": {
        "amount": 1,
        "parent_coin_info": "0xab2b05b64057883971197d524569f96313656ffe213d63879a0ce019505f0a60",
        "puzzle_hash": "0x7fb5cdb5f02bd956147cca3e4420dce2cac324d44840eae544a7f42e2c1ed124"
      },
      "coinbase": false,
      "confirmed_block_index": 1022620,
      "spent": false,
      "spent_block_index": 0,
      "timestamp": 1653467258
    }
  ],
  "success": true
}


In [ ]:
%%bash
# let change the pool and see how the PlotNFT changes


# References
- [Pooling FAQ](https://github.com/Chia-Network/chia-blockchain/wiki/Pooling-FAQ)
- **Chia Pools for Pool Operators**([video](https://www.youtube.com/watch?v=XzSZwxowPzw), [pdf](https://www.chia.net/assets/presentations/2021-06-02_Pooling_for_Pool_Operators.pdf))
- [Pool Protocol](https://docs.chia.net/docs/11pooling/pooling)
- [Pooling User Guide](https://github.com/Chia-Network/chia-blockchain/wiki/Pooling-User-Guide)
- [pool-reference](https://github.com/Chia-Network/pool-reference)
- [CLI](https://github.com/Chia-Network/chia-blockchain/wiki/CLI-Commands-Reference#plots)